In [1]:
<script src="https://unpkg.com/requirejs@2.3.6/bin/r.js"></script>
<script type="module">
	import * as lh from 'https://unpkg.com/lit-html@1.3.0/lit-html.js';
	
require.config({
        paths: {
            "igniteui-webcomponents-charts": 'https://unpkg.com/igniteui-webcomponents-charts@1.2.1/bundles/igniteui-webcomponents-charts.umd',
            "igniteui-webcomponents-grids": 'https://unpkg.com/igniteui-webcomponents-charts@1.2.1/bundles/igniteui-webcomponents-grids.umd',
            "igniteui-webcomponents-core": 'https://unpkg.com/igniteui-webcomponents-core@1.2.1/bundles/igniteui-webcomponents-core.umd',
            "tslib": 'https://unpkg.com/tslib@2.1.0/tslib'
        }
      }
    );
    
    define("lit-html", [], function () {
      return lh;
    });
    	 
    require(["igniteui-webcomponents-core", "igniteui-webcomponents-charts", "igniteui-webcomponents-grids"], function (igCore, igCharts, igGrids) {
      window.igCore = igCore;
      window.igCharts = igCharts;
      window.igGrids = igGrids
      igCore.ModuleManager.register(
        igCharts.IgcCategoryChartModule,
        igGrids.IgcDataGridModule
      );
    });    
   
</script>

Next we create a helper method that aids in creating charts.

In [1]:
notebookScope.displayChart = function(data, element, options) {
    //data = JSON.parse(data);
    if (options["extractTitle"]) {
        for (var i = 0; i < data.length; i++) {
            if (data[i].length > 0) {
                data[i].title = data[i][0][options["extractTitle"]];            
            }
        }
        
    }
    var chart = document.createElement("igc-category-chart");
    chart.style.margin = "30px";
    chart.width = "450px";
    chart.height = "300px";
    for (var p in options) {
        chart[p] = options[p];
    }
    chart.dataSource = data;
    
    element.append(chart);
}

Next we create a helper method that aids in creating grids.

In [1]:
notebookScope.displayGrid = function(data, element, options) {
    var flattened = [];
    //data = JSON.parse(data);
    if (data.length > 0 && data[0].length > 0) {
        for (var i = 0; i < data.length; i++) {
            for (var j = 0; j < data[i].length; j++) {
                flattened.push(data[i][j]);
            }
        }
    }

    var grid = document.createElement("igc-data-grid");
    grid.width = "450px";
    grid.height = "300px";
    
    for (var p in options) {
        grid[p] = options[p];
    }
    grid.dataSource = flattened;
    
    element.append(grid);
}

Next we download and process this [Covid Data](https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv) and display the deaths by state.

In [1]:
public class CovidData
{
    public string State { get; set; }
    public double Cases { get; set; }
    public double Deaths { get; set; }
    public string Date { get; set; }
}

In [1]:
#r "nuget:CsvHelper"
using CsvHelper;
using CsvHelper.Configuration;
using System.IO;
using System.Globalization;
using System.Collections.Generic;
using System.Net;
using System.Linq;
using System.Text.Json;
using System.Text.Json.Serialization;

IEnumerable<CovidData> records;
HttpWebRequest req = (HttpWebRequest)WebRequest.Create("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv");
HttpWebResponse resp = (HttpWebResponse)req.GetResponse();

using (StreamReader sr = new StreamReader(resp.GetResponseStream()))
{
    var config = new CsvConfiguration(CultureInfo.InvariantCulture)
    {
        PrepareHeaderForMatch = args => args.Header.ToLower(),
    };
    using (var csv = new CsvReader(sr, config))
    {
        records = csv.GetRecords<CovidData>().ToList();
    }
}

var allDates = (from item in records
                    orderby item.Date
                    select item.Date);
var datesSet = new HashSet<string>();
foreach (var date in allDates) { datesSet.Add(date); }

var byState = (from item in records
                orderby item.Date
                where item.State != "District of Columbia" && item.State != "Virgin Islands" &&
                item.State != "Guam" && item.State != "Northern Mariana Islands" &&
                item.State != "Puerto Rico"
                group item by item.State into g
                select g.ToList());

var filledInStates = new List<List<CovidData>>();
foreach (var stateData in byState) 
{
    var state = stateData.First().State;
    var filledIn = (from date in datesSet
        join stateItem in stateData on date equals stateItem.Date into lj
        from subItem in lj.DefaultIfEmpty()
        select new CovidData() { 
            Date = date, 
            State = state, 
            Deaths = (subItem != null ? subItem.Deaths : 0),
            Cases = (subItem != null ? subItem.Cases : 0) 
            }).ToList();
    filledInStates.Add(filledIn);
}


In [1]:
<div id="chartDeaths" style="background-color: white; float: left"></div>

#!javascript

var data = interactive.csharp.getVariable("filledInStates");
data.then((d) => {
    notebookScope.displayChart(d, document.getElementById("chartDeaths"), { chartTitle: 'covid deaths by state', excludedProperties: ['*Cases', '*Fips'], extractTitle: 'State' })
});

In [1]:
<div id="chartCases" style="background-color: white; float: left"></div>

#!javascript

var data = interactive.csharp.getVariable("filledInStates");
data.then((d) => {
    notebookScope.displayChart(d, document.getElementById("chartCases"), { chartTitle: 'covid cases by state', excludedProperties: ['*Deaths', '*Fips'], extractTitle: 'State' })
});

In [1]:
<div id="gridData" style="background-color: white; float: left"></div>

#!javascript

var data = interactive.csharp.getVariable("filledInStates");
data.then((d) => {
    notebookScope.displayGrid(d, document.getElementById("gridData"), { defaultColumnMinWidth: 150 })
});